## 20200901T22:00 Anaconda 和 PyTorch 安装配置
被JHU Data Science课虐的头昏脑涨，决定换换脑子，回头玩Jetson Nano，再向前推进一步。核心思路是，在Nano板子上配置好python，Anaconda和PyTorch环境。

**如无特别指明，以下命令都是在ssh登录到jetson nano后远程执行。**

<img src="./Figures/517720653.png" width = 600>

### oh-my-zsh (optional)
用zsh替代默认的bash，并安装oh-my-zsh
```shell
$ apt install curl
$ sh -c "$(curl -fsSL https://raw.github.com/ohmyzsh/ohmyzsh/master/tools/install.sh)"
```

### python

```shell
$ sudo apt install python3-pip python3-dev libhdf5-serial-dev hdf5-tools libhdf5-dev zlib1g-dev zip libjpeg8-dev
$ pip3 -V # 查看pip3的版本
pip 9.0.1 from /usr/lib/python3/dist-packages (python 3.6)

```

首先将pip版本从9.0.1升级到最新的20.2.2，升级之后可以更换为清华源，提高访问速度 <a href="https://mirrors.tuna.tsinghua.edu.cn/help/pypi/">https://mirrors.tuna.tsinghua.edu.cn/help/pypi/</a>

```shell
$ python3 -m pip install --upgrade pip -i https://pypi.tuna.tsinghua.edu.cn/simple # 临时使用1次tsinghua源升级pip，提高速度
$ pip3 config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple # 永久换成tsinghua源
````


### Anaconda

Anaconda安装比较简单，不赘述了。以下简要记述安装过程。

需要注意的是，Jetson Nano的CPU是aarch64架构，对应地，Anaconda有针对aarch64架构的发行版，叫做Archiconda，这是github上的Archiconda主页，网址在<a href="https://github.com/Archiconda/build-tools">https://github.com/Archiconda/build-tools</a>，感兴趣的不妨上去看看。

#### 安装Archiconda

下载
```shell
$ mkdir ~/Soft
$ cd ~/Soft
$ wget https://github.com/Archiconda/build-tools/releases/download/0.2.3/Archiconda3-0.2.3-Linux-aarch64.sh
$ chmod +x ./Archiconda3-0.2.3-Linux-aarch64.sh
$ sh ./Archiconda3-0.2.3-Linux-aarch64.sh
```
安装

```shell
$ sudo ./Archiconda3-0.2.3-Linux-aarch64.sh
```

改成清华源，提高访问速度
```shell
$ touch ~/.condarc
$ vim ~/.condarc
```
填入如下内容(来自<a href="https://mirrors.tuna.tsinghua.edu.cn/help/anaconda/">https://mirrors.tuna.tsinghua.edu.cn/help/anaconda/</a>)
```shell
channels:
  - defaults
show_channel_urls: true
channel_alias: https://mirrors.tuna.tsinghua.edu.cn/anaconda
default_channels:
  - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
  - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/free
  - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/r
  - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/pro
  - https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/msys2
custom_channels:
  conda-forge: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud
  msys2: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud
  bioconda: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud
  menpo: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud
  pytorch: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud
  simpleitk: https://mirrors.tuna.tsinghua.edu.cn/anaconda/cloud
```

清除索引缓存，确保清华源生效
```shell
$ conda clean -i
```

然后就是建立一个名为work的虚拟环境，可以在里面随意折腾了
```shell
$ conda update -n base conda #update最新版的conda
$ conda create -n work python=3.6 #可以将work改为自己想要的名字
$ conda activate work
```

### PyTorch & torchvision

Jetson Nano在官方image中已经预集成了GPU驱动，<a href="https://forums.developer.nvidia.com/t/nvidia-driver-not-included-in-the-image-for-jetson-nano/76795">不需要额外下载nvidia-driver-430等PCIe驱动<a>

NVIDIA官方提供了<a href = "https://forums.developer.nvidia.com/t/pytorch-for-jetson-nano-version-1-6-0-now-available/72048">Jetson Nano的PyTorch 1.6.0和torchvision的下载<a>，照着操作就行了。注意：JetPack 4.4 (L4T R32.4.3)只支持$pyTorch 1.6.0 or newer (due to updates in cuDNN).

首先是`pytorch`

```shell
$ cd ~/Soft
$ wget https://nvidia.box.com/shared/static/9eptse6jyly1ggt9axbja2yrmj6pbarc.whl -O torch-1.6.0-cp36-cp36m-linux_aarch64.whl
$ sudo apt-get install python3-pip libopenblas-base libopenmpi-dev 
$ conda install Cython
$ conda install numpy 
$ pip3 install ./torch-1.6.0-cp36-cp36m-linux_aarch64.whl
```

然后是`torchvision`。注意：与PyTorch 1.6.0对应的torchvision版本是0.8.0
```shell
$ sudo apt install libjpeg-dev zlib1g-dev
$ git clone --branch 0.8.0 https://github.com/pytorch/vision torchvision
$ cd torchvision
$ export BUILD_VERSION=0.8.0 # 写入.zshrc或者.bashrc的环境变量中
$ sudo python3 setup.py install
```

测试一下吧
```shell
$ python3.6
Python 3.6.9 (default, Jul 17 2020, 12:50:27) 
[GCC 8.4.0] on linux
Type "help", "copyright", "credits" or "license" for more information.
>>> import torch
>>> print(torch.__version__)
1.6.0
>>> print('CUDA available: ' + str(torch.cuda.is_available()))
CUDA available: True
>>> print('cuDNN version: ' + str(torch.backends.cudnn.version()))
cuDNN version: 8000
>>> a = torch.cuda.FloatTensor(2).zero_()
>>> print('Tensor a = ' + str(a))
Tensor a = tensor([0., 0.], device='cuda:0')
>>> b = torch.randn(2).cuda()
>>> print('Tensor b = ' + str(b))
Tensor b = tensor([-1.3437,  0.1451], device='cuda:0')
>>> c = a + b
>>> print('Tensor c = ' + str(c))
Tensor c = tensor([-1.3437,  0.1451], device='cuda:0')
```

```shell
$ python3.6
>>> import torchvision
>>> print(torchvision.__version__)
0.8.0a0+5f616a2
```

### Next Steps
1. 组装jetbot小车的硬件
2. 配置，让nano能够指挥jetbot小车动起来